# Integration Tests
This notebook demonstrates running agents via the registry with a stub engine for testing.

In [ ]:
import os, sys, types
from pathlib import Path
from app.utils.agent_factory import build_agent
from app.utils.symbol_graph.symbol_service import SymbolService

# Fake engine to avoid OpenAI dependency
class FakeEngine:
    def __init__(self):
        self.calls = []
    def call_engine(self, prompt: str, config: dict):
        self.calls.append(prompt)
        return '# transformed code'

fake_mod = types.SimpleNamespace(OpenAiGpt4oAgentEngineRunner=FakeEngine)
sys.modules['app.utils.engine.openai_gpt4o_runner'] = fake_mod
os.environ['OPENAI_API_KEY'] = 'fake'

symbol_service = SymbolService(Path('.'))
log_path = Path('integration_logs/prompt_log.jsonl')
log_path.parent.mkdir(exist_ok=True)

## Test 1: Generator Agent
Run the generator agent on a sample file.

In [ ]:
agent = build_agent('generator', symbol_service)
result1 = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': '1', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('Generator output:', result1['response'][:60])

## Test: discriminator agent

In [ ]:
agent = build_agent('discriminator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'discriminator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('discriminator result:', res['response'][:40])

## Test: mediator agent

In [ ]:
agent = build_agent('mediator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'mediator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('mediator result:', res['response'][:40])

## Test: patchor agent

In [ ]:
agent = build_agent('patchor', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'patchor', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('patchor result:', res['response'][:40])

## Test: recommender agent

In [ ]:
agent = build_agent('recommender', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('recommender result:', res['response'][:40])

## Test: generator agent

In [ ]:
agent = build_agent('generator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'generator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('generator result:', res['response'][:40])

## Test: discriminator agent

In [ ]:
agent = build_agent('discriminator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'discriminator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('discriminator result:', res['response'][:40])

## Test: mediator agent

In [ ]:
agent = build_agent('mediator', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'mediator', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('mediator result:', res['response'][:40])

## Test: patchor agent

In [ ]:
agent = build_agent('patchor', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'patchor', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('patchor result:', res['response'][:40])

## Test: recommender agent

In [ ]:
agent = build_agent('recommender', symbol_service)
res = agent.run({'symbol': 'my_symbol', 'file': 'tests/data/test_context_provider_preprocessor/example.py', 'experiment_id': 'demo', 'run_id': 'recommender', 'round': 1, 'system': 'demo', 'log_path': log_path})
print('recommender result:', res['response'][:40])

## Log Verification

In [ ]:
import json
lines=[json.loads(l) for l in log_path.read_text().splitlines()]
print('entries:', len(lines))
print({e['agent_role'] for e in lines})

## Basic Analysis

In [ ]:
from collections import Counter
counts=Counter(e['agent_role'] for e in lines)
print(counts)

In [ ]:
for e in lines[:2]:
    print(e['agent_role'], '->', e['response'][:30])